In [1]:
import urllib.request
import string, math, time, collections, pprint
import json
import pandas as pd
import nltk.corpus
import nltk.tokenize
import numpy as np
import keras
import matplotlib.pyplot as plt
import sklearn
import gensim
import os
import nltk, re, pprint
import textblob
import stanfordcorenlp

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
#This takes the json structure from the pushshift api
with urllib.request.urlopen('https://api.pushshift.io/reddit/search/submission/?subreddit=BirthControl&metadata=true&after=30&size=500') as response:
    file = response.read()

#Here we're using the json package to convert the json content into a dictionary.
#The API returns a dictionary with one element (key 'data') which is a list of dictionaries. We can access this list of dictionaries with parsed_json['data']    
parsed = json.loads(file)
parsed

# #Using pandas, we are taking the list of dictionries and converting it into a DataFrame, and then exporting the DataFrame as a csv file.
# BC_dataframe_uncleaned = pd.DataFrame(parsed['data'])
# BC_dataframe_uncleaned.to_csv('database.csv')

# BC_dataframe_uncleaned

{'data': [{'author': '[deleted]',
   'author_flair_css_class': None,
   'author_flair_text': None,
   'created_utc': 1311215507,
   'domain': 'self.birthcontrol',
   'full_link': 'https://www.reddit.com/r/birthcontrol/comments/ivfpe/do_you_need_condoms_or_another_method_of_birth/',
   'id': 'ivfpe',
   'is_self': True,
   'num_comments': 23,
   'over_18': False,
   'permalink': '/r/birthcontrol/comments/ivfpe/do_you_need_condoms_or_another_method_of_birth/',
   'score': 24,
   'selftext': "Myself and others would like to ensure that our fellow redditors are taken care of in terms of STD and pregnancy protection.\n\nIf you are willing to donate condoms, dental dams, pregnancy tests, etc to fellow redditors, please post here! Once you have donated to your capability, please reply to your thread letting us know so we don't continue to ask you for assistance.\n\nIf you are looking for help with birth control/STD prevention options, please PM one of the redditors here that is offering to se

In [3]:
# Create a dataframe with just the title and body

uncleaned = BC_dataframe_uncleaned[['title', 'selftext']].copy()
uncleaned = uncleaned.fillna("No text available")
new = pd.DataFrame(uncleaned)
new

,title,selftext
0,Do you need condoms or another method of birth...,Myself and others would like to ensure that ou...
1,50 pregnancy tests for $20.,No text available
2,Want to switch off of the pill,I have a notoriously bad memory and am always ...
3,Useful links for navigating the world of BC.,Here's a short list of the important links tha...
4,Hooray for no copays on birth control!,No text available
5,Beginning the 4 period a year birth control plan,So it's my first time on birth control pills (...
6,Acne & birth control,I've had issues with birth control and acne in...
7,Need some advice...,Sorry this got a bit long I just wanted to giv...
8,Non-hormonal birth control?{X-post to 2X},[Crosspost to 2X because I think it might get ...
9,"TwoX's big, bad birth control thread from back...",No text available


In [4]:
stop_words = set(stopwords.words('english'))
keepwords = ['against', 'ain', "aren't", 'but', "couldn't", 'couldn', "didn't", 'doesn'
                 , "doesn't", 'don', "don't", 'hadn', "hadn't", 'hasn', "hasn't"
                 , 'haven', "haven't", 'isn', "isn't", 'll', 'mightn', "mightn't"
                 , 'mustn', "mustn't", 'needn', "needn't", 'no', "not", "nor", "only"
                 , "shan", "shan't", "shouldn't", "should've", "shouldn", "wasn", "wasn't"
                 , "weren", "weren't", "won't", "wouldn", "wouldn't"]

for keepword in keepwords:
    if keepword in stop_words:
        stop_words.remove(keepword)

#Standardizing the original title and selftext
def standardize_text(new, text_field):
    new[text_field] = new[text_field].str.replace(r"http\S+", "")
    new[text_field] = new[text_field].str.replace(r"http", "")
    new[text_field] = new[text_field].str.replace(r"@\S+", "")
    new[text_field] = new[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    new[text_field] = new[text_field].str.replace(r"@", "at")
    new[text_field] = new[text_field].str.lower()
    return new

new = standardize_text(new, "title")
new = standardize_text(new, "selftext")
new

,title,selftext
0,do you need condoms or another method of birth...,myself and others would like to ensure that ou...
1,50 pregnancy tests for $20.,no text available
2,want to switch off of the pill,i have a notoriously bad memory and am always ...
3,useful links for navigating the world of bc.,here's a short list of the important links tha...
4,hooray for no copays on birth control!,no text available
5,beginning the 4 period a year birth control plan,so it's my first time on birth control pills (...
6,acne & birth control,i've had issues with birth control and acne in...
7,need some advice...,sorry this got a bit long i just wanted to giv...
8,non-hormonal birth control?{x-post to 2x},[crosspost to 2x because i think it might get ...
9,"twox's big, bad birth control thread from back...",no text available


In [78]:
##Gathering sentiments around particular sentences in a post

body = new["selftext"]
text = body[23]

sentence_tokenizer = textblob.tokenizers.SentenceTokenizer()
sentences = sentence_tokenizer.tokenize(text)

## Attach sentiment to each sentence
sentblobs = [textblob.TextBlob(sentence) for sentence in sentences]
[blob.sentiment for blob in sentblobs]

TypeError: unhashable type: 'list'

In [32]:
contraception = ['ocella', 'yasmin'] #create dictionary of birth control names in SQL

In [33]:
## This tells you which drugs are mentioned in which sentences

mentions = []
for sent in sentences:
    sentmap = []
    for i, contra in enumerate(contraception):
        if contra in sent.lower():
            sentmap.append(i)
    mentions.append(sentmap)
mentions

[[0, 1], [0], [], [], [], [], [], [], []]

In [34]:
contratexts = {}
lastdrug = 'overall'
for i,sent in enumerate(sentblobs):
    dms = mentions[i]
    if len(dms) == 1:
        lastdrug = contraception[dms[0]]
    if lastdrug in contratexts.keys():
        contratexts[lastdrug].append(sent)
    else:
        contratexts[lastdrug] = [sent]

In [35]:
contratexts

{'ocella': [TextBlob("then my pharmacy refills me with something else and it doesn't work, after two months i talk to my pharmacist and they say no they still have ocella despite rumors of it being pulled off the market and start refilling me with that again."),
  TextBlob("so... it stops working."),
  TextBlob("after four months of two week long periods i went in and got switched to orthotricyclin."),
  TextBlob("i have now taking the very last pill in my first pack- and i have had no period at all."),
  TextBlob("when i first started taking it hubby and i were doing it like crazy- this pill seemed to turn me into a hormonal teenager all over again."),
  TextBlob("so now... the question is could i be pregnant or is this just a normal every day case of first month on new pill."),
  TextBlob("i know i know- go piss on a stick."),
  TextBlob("but... any opinions and experiences to share in the meantime?")],
 'overall': [TextBlob("so, i was on ocella (generic for yasmin) for about eight y

In [40]:
contratexts_merged = {}
for contra in contratexts.keys():
    textblobs = contratexts[contra]
    mergedblob = textblob.TextBlob('')
    for blob in textblobs:
        mergedblob += blob
    contratexts_merged[contra] = mergedblob

then my pharmacy refills me with something else and it doesn't work, after two months i talk to my pharmacist and they say no they still have ocella despite rumors of it being pulled off the market and start refilling me with that again.so... it stops working.after four months of two week long periods i went in and got switched to orthotricyclin.i have now taking the very last pill in my first pack- and i have had no period at all.when i first started taking it hubby and i were doing it like crazy- this pill seemed to turn me into a hormonal teenager all over again.so now... the question is could i be pregnant or is this just a normal every day case of first month on new pill.i know i know- go piss on a stick.but... any opinions and experiences to share in the meantime?


In [38]:
for contra in contratexts_merged:
    print(contratexts_merged[contra],contratexts_merged[contra].sentiment)
    
##This doesn't do a very good job of analyzing sentiments for each sentence, but is alright for now. Also, 
##the sentence referring to "ocella" alone doesn't show up. Why?

so, i was on ocella (generic for yasmin) for about eight years for heavy bleeding and severe cramps. Sentiment(polarity=-0.1, subjectivity=0.25)
then my pharmacy refills me with something else and it doesn't work, after two months i talk to my pharmacist and they say no they still have ocella despite rumors of it being pulled off the market and start refilling me with that again.so... it stops working.after four months of two week long periods i went in and got switched to orthotricyclin.i have now taking the very last pill in my first pack- and i have had no period at all.when i first started taking it hubby and i were doing it like crazy- this pill seemed to turn me into a hormonal teenager all over again.so now... the question is could i be pregnant or is this just a normal every day case of first month on new pill.i know i know- go piss on a stick.but... any opinions and experiences to share in the meantime? Sentiment(polarity=0.07996632996632996, subjectivity=0.44346801346801346)


In [ ]:
#Counting words in title and body
title_words = [word for entry in new["title_clean"] for word in entry]
body_words = [word for entry in new["body_clean"] for word in entry]

title_lengths = [len(entry) for entry in new["title_clean"]]
body_lengths = [len(entry) for entry in new["body_clean"]]

VOCAB_title = sorted(list(set(title_words)))
VOCAB_body = sorted(list(set(body_words)))

print("%s words total, with a vocabulary size of %s" % (len(title_words), len(VOCAB_title)))
print("Max sentence length is %s" % max(title_lengths))

print("%s words total, with a vocabulary size of %s" % (len(body_words), len(VOCAB_body)))
print("Max sentence length is %s" % max(body_lengths))

In [ ]:
#Plotting length of titles
fig = plt.figure(figsize=(10, 10)) 
plt.xlabel('Title Sentence length')
plt.ylabel('Number of sentences in title')
plt.hist(title_lengths)
plt.show()

In [ ]:
#Plotting length of titles
fig = plt.figure(figsize=(10, 10)) 
plt.xlabel('Body Sentence length')
plt.ylabel('Number of sentences in Body')
plt.hist(body_lengths)
plt.show()

In [ ]:
#Separating Title and body 
title_corpus = for_bow2["title"].tolist()
body_corpus = for_bow2["selftext"].tolist()

#new1 = pd.DataFrame[{"title_corpus":title_corpus,'body_corpus':body_corpus}]

#Creating Bag Of Words
def cv(data):
    count_vectorizer = CountVectorizer()
    
    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

new_title_counts, count_vectorizer = cv(title_corpus)
new_body_counts, count_vectorizer = cv(body_corpus)

bow_title = pd.DataFrame(new_title_counts.toarray())
bow_body = pd.DataFrame(new_body_counts.toarray())

In [ ]:
#SENTIMENT ANALYSIS ON UNCLEANED BODY

sia = SIA()
results_body = []
results_title = []

for entry in new["selftext"]:
    body_score = sia.polarity_scores(entry)
    body_score['selftext'] = entry
    results_body.append(body_score)
new_results = pd.DataFrame(results_body)

for entry in new["title"]:
    title_score = sia.polarity_scores(entry)
    title_score['title'] = entry
    results_title.append(title_score)
results_title = pd.DataFrame(results_title)

##Change column names
new_results = new_results.rename(columns={'compound': 'body_comp', 'neg': 'body_neg', 'neu': 'body_neu', 'pos':'body_pos'})
results_title = results_title.rename(columns={'compound': 'title_comp', 'neg': 'title_neg', 'neu': 'title_neu', 'pos':'title_pos'})

bigdata = new_results.join(results_title)

#Counting posts as positive/negative
label = []
body_comp_label = bigdata['body_comp'] 
title_comp_label = bigdata['title_comp']

for n in range(len(body_comp_label)):
    if (body_comp_label[n] > 0.3) and (title_comp_label[n] > 0.3):
        label.append(1)
    elif (body_comp_label[n] < -0.3) and (title_comp_label[n] < -0.3):
        label.append(-1)
    else:
        label.append(0)

bigdata['composite_label'] = label

bigdata['body_label'] = 0
bigdata.loc[bigdata['body_comp'] > 0.3, 'body_label'] = 1
bigdata.loc[bigdata['body_comp'] < -0.3, 'body_label'] = -1
bigdata.head(50)

#bigdata.to_csv('Data.6.13.18.csv')

In [ ]:
#Subsetting dataframe based on the user's input (for now, IUD)

IUD = bigdata[bigdata['selftext'].str.contains("iud")]
IUD = pd.DataFrame(IUD)
IUD.to_csv('IUD_Data.6.13.18.csv')
IUD

In [ ]:
names = ['Positive', 'Negative', 'Neutral']
colors = ['green', 'red', 'orange']
counts = IUD.body_label.value_counts(normalize=True)*100

plt.pie(counts, labels = names, colors=colors, startangle=90, autopct = '%.1f%%')
plt.show()